In [ ]:
from palmerpenguins import penguins
from pandas import get_dummies
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

## Get Data


In [ ]:
# df = penguins.load_penguins().dropna() - instead of loading the data from the palmer penguins module, we can load it from the duck db we have created

import duckdb

con = duckdb.connect("my-db.duckdb")
df = con.execute("SELECT * FROM penguins").fetchdf().dropna()
con.close()

# df.head(3)

## Define Model and Fit


In [ ]:
X = get_dummies(df[['bill_length_mm', 'species', 'sex']], drop_first = True)
y = df['body_mass_g']

model = LinearRegression().fit(X, y)

## Get some information


In [ ]:
print(f"R^2 {model.score(X,y)}")
print(f"Intercept {model.intercept_}")
print(f"Columns {X.columns}")
print(f"Coefficients {model.coef_}")

## Save the model so it can be placed outside the bundle

In [ ]:
# take processing function/business logic out of the presentation layer

# create VetiverModel class object
from vetiver import VetiverModel
v = VetiverModel(model, model_name = "penguin_model", prototype_data = X)
# model = a trained model
# model_name = a name you want to assign to the model
# prototype_data = a sample of the data


# save model
from pins import board_folder
from vetiver import vetiver_pin_write
model_board = board_folder("data/model/", allow_pickle_read = True) # use a local folder as a board
vetiver_pin_write(model_board, v) # write the model into a board

## Create an API for model predictions

In [ ]:
# open model if needed
# b = board_folder("data/model", allow_pickle_read = True)
# v = VetiverModel.from_pin(b, "penguin_model")

# auto-generate a fastAPI from the model
from vetiver import VetiverAPI
app = VetiverAPI(v, check_prototype = True)

# run app
app.run(port = 8080, host = "127.0.0.1") # view API with http://localhost:8080